In [2]:
import os
import pandas as pd
from keybert import KeyBERT

In [3]:
# Load Data
df_patents = pd.read_csv('/Users/juergenthiesen/Documents/Patentsview/Cleantech CSV Files/g_patent_Y02.csv')
df_cpc = pd.read_csv('/Users/juergenthiesen/Documents/Patentsview/Cleantech CSV Files/g_cpc_current_Y02.csv')

# Drop all classes uneqaul to Y02 from df_cpc
df_cpc = df_cpc[df_cpc['cpc_class'].str.contains('Y02')]

# Merge data on patent_id, keep duplicates
df = pd.merge(df_patents, df_cpc, on='patent_id', how='left')

# Drop all data in df with title or abstract NaN
df = df.dropna(subset=['patent_title', 'patent_abstract'])

# Merge title and abstract with [SEP] token
df['patent_title_abstract'] = df['patent_title'] + ' [SEP] ' + df['patent_abstract']

# Randomly sample 1000 patents per cpc_subclass
df = df.groupby('cpc_subclass').apply(lambda x: x.sample(1000, random_state=42))

In [4]:
# Transform dataframe column 'patent_title_abstract' into a list of strings
patent_title_abstract = df['patent_title_abstract'].tolist()

# Extract keywords from patent_title_abstract
kw_extractor = KeyBERT('climatebert/distilroberta-base-climate-f')

No sentence-transformers model found with name /Users/juergenthiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /Users/juergenthiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from

In [5]:
keywords = kw_extractor.extract_keywords(patent_title_abstract, keyphrase_ngram_range=(1, 2), stop_words=None)